# **Directory**

In [2]:
import numpy as np 
import pandas as pd

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input/aptos2019-blindness-detection/'):
    print(dirname)

# **Visualizing Training Data**

In [4]:
BASE_PATH='/root/dalishi/analysis'
train_dataset=pd.read_csv(os.path.join(BASE_PATH,'train_data.csv'))
test_dataset=pd.read_csv(os.path.join(BASE_PATH,'test_data.csv'))

In [5]:
train_dataset.head(3)

,图片路径,标签
0,b'/root/dalishi/small_images/5131.jpg',16
1,b'/root/dalishi/small_images/10395.jpg',134
2,b'/root/dalishi/small_images/7249.jpg',65


In [6]:
test_dataset.head(3)

,图片路径,标签
0,b'/root/dalishi/small_images/9.jpg',0
1,b'/root/dalishi/small_images/13.jpg',0
2,b'/root/dalishi/small_images/17.jpg',0


In [7]:
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

## ****Importing and Installing Libraries****

In [8]:
!pip install --upgrade efficientnet-pytorch

Requirement already up-to-date: efficientnet-pytorch in /root/anaconda3/lib/python3.6/site-packages (0.7.0)


In [9]:
!pip install torchsummary

In [10]:
import PIL
import torch
from time import time
import torchvision
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torchsummary import summary
from torch.autograd import Variable
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet

## **Data-Loader**

In [11]:
class Dataset(data.Dataset):
    def __init__(self,csv_path,transform=None):
        self.train_set=pd.read_csv(csv_path)
        self.transform=transform
    def __len__(self):
        return len(self.train_set)
    
    def __getitem__(self,idx):
        file_name=self.train_set.iloc[idx][0]
        label=self.train_set.iloc[idx][1]
        img=Image.open(eval(file_name))
        if self.transform is not None:
            img=self.transform(img)
        return img,label

## **Defining Transforms and Parameters for Training**

In [12]:
params = {'batch_size': 16,
          'shuffle': True
         }
epochs = 100
learning_rate=1e-3

In [13]:
transform_train = transforms.Compose([transforms.Resize((224,224)),transforms.RandomApply([
        torchvision.transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip()],0.7),
		transforms.ToTensor()])

In [14]:
training_set=Dataset(os.path.join(BASE_PATH,'train_data.csv'),transform=transform_train)
training_generator=data.DataLoader(training_set,**params)

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

cuda:0


## **Importing the Model (Efficient Net)**

In [16]:
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=201)

Loaded pretrained weights for efficientnet-b7


In [17]:
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        64, 16, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        16, 64, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        64, 32, kernel_siz

In [18]:
print(summary(model, input_size=(3, 224, 224)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 64, 112, 112]           1,728
       BatchNorm2d-3         [-1, 64, 112, 112]             128
MemoryEfficientSwish-4         [-1, 64, 112, 112]               0
         ZeroPad2d-5         [-1, 64, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 64, 112, 112]             576
       BatchNorm2d-7         [-1, 64, 112, 112]             128
MemoryEfficientSwish-8         [-1, 64, 112, 112]               0
          Identity-9             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 16, 1, 1]           1,040
MemoryEfficientSwish-11             [-1, 16, 1, 1]               0
         Identity-12             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 64, 1, 1]           1,088
         I

In [19]:
PATH_SAVE='./Weights/'
if(not os.path.exists(PATH_SAVE)):
    os.mkdir(PATH_SAVE)

In [20]:
criterion = nn.CrossEntropyLoss()
lr_decay=0.99
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# **Training The Model**

In [21]:
eye = torch.eye(201).to(device)
classes=list(range(201))

In [22]:
history_accuracy=[]
history_loss=[]
epochs = 25

In [ ]:
for epoch in range(epochs):  
    running_loss = 0.0
    correct=0
    total=0
    class_correct = list(0. for _ in classes)
    class_total = list(0. for _ in classes)
    for i, data in enumerate(training_generator, 0):
        inputs, labels = data
        t0 = time()
        inputs, labels = inputs.to(device), labels.to(device)
        labels = eye[labels]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, torch.max(labels, 1)[1])
        _, predicted = torch.max(outputs, 1)
        _, labels = torch.max(labels, 1)
        c = (predicted == labels.data).squeeze()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        accuracy = float(correct) / float(total)
        
        history_accuracy.append(accuracy)
        history_loss.append(loss)
        
        loss.backward()
        optimizer.step()
        
        for j in range(labels.size(0)):
            label = labels[j]
            class_correct[label] += c[j].item()
            class_total[label] += 1
        
        running_loss += loss.item()
        
        print( "Epoch : ",epoch+1," Batch : ", i+1," Loss :  ",running_loss/(i+1)," Accuracy : ",accuracy,"Time ",round(time()-t0, 2),"s" )
    for k in range(len(classes)):
        if(class_total[k]!=0):
            print('Accuracy of %5s : %2d %%' % (classes[k], 100 * class_correct[k] / class_total[k]))
        
    print('[%d epoch] Accuracy of the network on the Training images: %d %%' % (epoch+1, 100 * correct / total))
        
    if epoch%10==0 or epoch==0:
        torch.save(model.state_dict(), os.path.join(PATH_SAVE,str(epoch+1)+'_'+str(accuracy)+'.pth'))
        


Epoch :  1  Batch :  1  Loss :   5.327298641204834  Accuracy :  0.0 Time  0.64 s
Epoch :  1  Batch :  2  Loss :   5.2383713722229  Accuracy :  0.125 Time  0.58 s
Epoch :  1  Batch :  3  Loss :   5.079680919647217  Accuracy :  0.1875 Time  0.58 s
Epoch :  1  Batch :  4  Loss :   4.8715033531188965  Accuracy :  0.25 Time  0.58 s
Epoch :  1  Batch :  5  Loss :   4.704757976531982  Accuracy :  0.2875 Time  0.58 s
Epoch :  1  Batch :  6  Loss :   4.525586048762004  Accuracy :  0.3229166666666667 Time  0.58 s
Epoch :  1  Batch :  7  Loss :   4.523797988891602  Accuracy :  0.30357142857142855 Time  0.58 s
Epoch :  1  Batch :  8  Loss :   4.399340867996216  Accuracy :  0.328125 Time  0.58 s
Epoch :  1  Batch :  9  Loss :   4.271479262246026  Accuracy :  0.3402777777777778 Time  0.58 s
Epoch :  1  Batch :  10  Loss :   4.238374805450439  Accuracy :  0.3375 Time  0.58 s
Epoch :  1  Batch :  11  Loss :   4.109132268212059  Accuracy :  0.35795454545454547 Time  0.58 s
Epoch :  1  Batch :  12  Loss

Epoch :  1  Batch :  88  Loss :   2.641458114439791  Accuracy :  0.5056818181818182 Time  0.58 s
Epoch :  1  Batch :  89  Loss :   2.6312339037991643  Accuracy :  0.5077247191011236 Time  0.58 s
Epoch :  1  Batch :  90  Loss :   2.6280965540144177  Accuracy :  0.5083333333333333 Time  0.58 s
Epoch :  1  Batch :  91  Loss :   2.6224154954428203  Accuracy :  0.5089285714285714 Time  0.58 s
Epoch :  1  Batch :  92  Loss :   2.618714055289393  Accuracy :  0.5101902173913043 Time  0.58 s
Epoch :  1  Batch :  93  Loss :   2.6134932861533215  Accuracy :  0.5120967741935484 Time  0.58 s
Epoch :  1  Batch :  94  Loss :   2.610196719778345  Accuracy :  0.5126329787234043 Time  0.58 s
Epoch :  1  Batch :  95  Loss :   2.602534663049798  Accuracy :  0.5131578947368421 Time  0.58 s
Epoch :  1  Batch :  96  Loss :   2.592618121455113  Accuracy :  0.5143229166666666 Time  0.58 s
Epoch :  1  Batch :  97  Loss :   2.5819487829798278  Accuracy :  0.5167525773195877 Time  0.58 s
Epoch :  1  Batch :  98  

Epoch :  1  Batch :  172  Loss :   2.1586990664864696  Accuracy :  0.581031976744186 Time  0.58 s
Epoch :  1  Batch :  173  Loss :   2.1540396692435864  Accuracy :  0.5823699421965318 Time  0.58 s
Epoch :  1  Batch :  174  Loss :   2.1515154670709853  Accuracy :  0.5826149425287356 Time  0.58 s
Epoch :  1  Batch :  175  Loss :   2.152790467057909  Accuracy :  0.5817857142857142 Time  0.58 s
Epoch :  1  Batch :  176  Loss :   2.149810303002596  Accuracy :  0.5823863636363636 Time  0.58 s
Epoch :  1  Batch :  177  Loss :   2.141594058039498  Accuracy :  0.5836864406779662 Time  0.58 s
Epoch :  1  Batch :  178  Loss :   2.1394917352145977  Accuracy :  0.5839185393258427 Time  0.58 s
Epoch :  1  Batch :  179  Loss :   2.133167123328374  Accuracy :  0.585195530726257 Time  0.58 s
Epoch :  1  Batch :  180  Loss :   2.1274266762865914  Accuracy :  0.5861111111111111 Time  0.58 s
Epoch :  1  Batch :  181  Loss :   2.1215243079385706  Accuracy :  0.5870165745856354 Time  0.58 s
Epoch :  1  Batc

## **Visualizing The Training Accuracy and losses**

In [ ]:
plt.plot(history_accuracy)
plt.plot(history_loss)

# **Inference**

In [ ]:
model.load_state_dict(torch.load('/kaggle/input/efficient-net/Weights/21_0.9243582741671218.pth'))

In [ ]:
model.eval()

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(512),
                                      transforms.ToTensor(),
                                     ])

In [ ]:
def predict_image(image):
    image_tensor = test_transforms(image)
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [ ]:
submission=pd.read_csv(BASE_PATH+'sample_submission.csv')

In [ ]:
submission.head(3)

In [ ]:
submission_csv=pd.DataFrame(columns=['id_code','diagnosis'])

In [ ]:
IMG_TEST_PATH=os.path.join(BASE_PATH,'test_images/')
for i in range(len(submission)):
    img=Image.open(IMG_TEST_PATH+submission.iloc[i][0]+'.png')
    prediction=predict_image(img)
    submission_csv=submission_csv.append({'id_code': submission.iloc[i][0],'diagnosis': prediction},ignore_index=True)
    if(i%10==0 or i==len(submission)-1):
        print('[',32*'=','>] ',round((i+1)*100/len(submission),2),' % Complete')

In [ ]:
submission_csv.to_csv('submission.csv',index=False)